# Patching Columns based on values provided on given CSV

CSV Assumptions:

1. The email's CSV has the correct number of columns and does not leave anything out.

    ie If a table has 10 columns, all those columns are accounted for in the CSV and there is 0 instances of missing columns (ie 9/10 columns instead of 10/10 columns) 

    The number of columns is critical to this exercise and it must be accurate.

    

2. The DBQ01 tables represented in this exercise can be interchanged with DBP01 tables like for like.

Reference: DAP-419
https://apps.nrs.gov.bc.ca/int/jira/browse/DAP-419 


In [1]:
import os
os.chdir("..")
from config import * 

Environment: test
Base URL: https://nr-data-catalogue-test.apps.emerald.devops.gov.bc.ca/api/v1


Ok great, let's try a PATCH request to ADD a tag based on column index

In [2]:
# Table and column identifiers
table_id = "aff703b7-c7a4-4f04-a85d-b741e6f97fea"
column_index = 0  # As found from your previous query

# URL for the PATCH request
url = f"https://nr-data-catalogue-test.apps.emerald.devops.gov.bc.ca/api/v1/tables/{table_id}?fields=columns,tags"

# JSON body for adding a tag to the specific column
data = [
    {
        "op": "add",  # "add" since you want to append the new tag
        "path": f"/columns/4/tags",  # Use the column index
        "value": [
            {
                "tagFQN": "Data Security Classification.Public" , # Fully Qualified Name of the tag
                 "labelType": "Manual",  # Label type
                 "state": "Confirmed"  # Tag state
            }
        ]
    }
]


# Send the PATCH request to add the tag
response = requests.patch(url, headers=headers_patch, json=data)

# Output the response status and message
if response.status_code == 200:
    print("Tag added successfully!")
else:
    print(f"Failed to add tag: {response.status_code}")
    print(response.json())


Tag added successfully!


Now, let's remove that tag

In [3]:
# JSON body for adding a tag to the specific column
data = [
    {
        "op": "remove",  # "add" since you want to append the new tag
        "path": f"/columns/4/tags",  # Use the column index
        "value": [
            {
                "tagFQN": "Data Security Classification.Public" , # Fully Qualified Name of the tag
                 "labelType": "Manual",  # Label type
                 "state": "Confirmed"  # Tag state
            }
        ]
    }
]


# Send the PATCH request to add the tag
response = requests.patch(url, headers=headers_patch, json=data)

# Output the response status and message
if response.status_code == 200:
    print("Tag removed successfully!")
else:
    print(f"Failed to remove tag: {response.status_code}")
    print(response.json())


Tag removed successfully!


Previously, we added a tag based on table ID, let's try based on table FQN

In [4]:
# Table and column identifiers
table_fqn = "DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE"
column_index = 0  # As found from your previous query

# URL for the PATCH request
url_fqn = f"https://nr-data-catalogue-test.apps.emerald.devops.gov.bc.ca/api/v1/tables/name/{table_fqn}?fields=columns,tags"

# JSON body for adding a tag to the specific column
data = [
    {
        "op": "add",  # "add" since you want to append the new tag
        "path": f"/columns/3/tags",  # Use the column index
        "value": [
            {
                "tagFQN": "Data Security Classification.Public" , # Fully Qualified Name of the tag
                 "labelType": "Manual",  # Label type
                 "state": "Confirmed"  # Tag state
            }
        ]
    }
]

# Send the PATCH request to add the tag
response = requests.patch(url_fqn, headers=headers_patch, json=data)

# Output the response status and message
if response.status_code == 200:
    print("Tag added successfully!")
else:
    print(f"Failed to add tag: {response.status_code}")
    print(response.json())

Tag added successfully!


Ok great, let's try to add tags based on the information provided on a CSV. 
The information on the CSV is strategically formatted in a way that should make things easy for us.

In [2]:
# df = pd.read_csv('reference_csvs/all_cols.csv').drop('Unnamed: 0',axis=1)
df = get_most_recent_csv('tests')

Most recent CSV file: tests\test_tables_with_columns_20241213_085805.csv


AttributeError: 'str' object has no attribute 'head'

The CSV we just imported contains all the tables and columns available in the DEV instance of our OpenMetaData Data Catalog (100,000+ rows).

It contains a near empty 'Tags' column which we need to fill using the information provided from another CSV (a compilation of the email's CSVs)... 


In [3]:
# Let's give column indexes to the values on the spreadsheet, this will be important later on
df['index'] = df.groupby('Table Name').cumcount()
df.head(30)

,Table Name,env,Database,Schema,Table,Column,Tag,index
0,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,aac_adjustment_reason_code,Test Classification.Ignore this tag,0
1,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,description,Test Classification.Ignore this tag,1
2,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,effective_date,NaN,2
3,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,expiry_date,NaN,3
4,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,update_timestamp,NaN,4
5,dlhdev.lhousedev.fdw_ods_fta_replication.aac_a...,dlhdev,lhousedev,fdw_ods_fta_replication,aac_allocation_amount,aac_allocation_amount_id,NaN,0
6,dlhdev.lhousedev.fdw_ods_fta_replication.aac_a...,dlhdev,lhousedev,fdw_ods_fta_replication,aac_allocation_amount,aac_adjustment_reason_code,NaN,1
7,dlhdev.lhousedev.fdw_ods_fta_replication.aac_a...,dlhdev,lhousedev,fdw_ods_fta_replication,aac_allocation_amount,aac_allocation_period_id,NaN,2
8,dlhdev.lhousedev.fdw_ods_fta_replication.aac_a...,dlhdev,lhousedev,fdw_ods_fta_replication,aac_allocation_amount,allowable_area_type_code,NaN,3
9,dlhdev.lhousedev.fdw_ods_fta_replication.aac_a...,dlhdev,lhousedev,fdw_ods_fta_replication,aac_allocation_amount,allowable_cut_type_code,NaN,4


The formatting for both CSV's need to be consistent to help merge them, the next few steps will take care of that

In [4]:
# Filter rows where the column contains alphabetic characters that are only uppercase (ignores non-alphabetic)
uppercase_only_df = df[df['Column'].apply(lambda x: any(c.isupper() for c in x) and all(not c.islower() for c in x))]

# Display the result
pd.set_option('display.max_rows', None)
uppercase_only_df

,Table Name,env,Database,Schema,Table,Column,Tag,index
7616,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_G,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_G,SUBSCRIBER#,NaN,1
7618,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_G,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_G,ADDRESS#,NaN,3
7622,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_H,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_H,SUBSCRIBER#,NaN,1
7624,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_H,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_H,ADDRESS#,NaN,3
7632,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_I,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_I,SUBSCRIBER#,NaN,0
7634,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_I,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_I,QUEUE#,NaN,2
7636,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_I,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_I,SENDER#,NaN,4
7637,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_I,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_I,TXN_STEP#,NaN,5
7647,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_L,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_L,SUBSCRIBER#,NaN,1
7649,ENV Test Database.ENVTEST1.pscis.AQ$_PSCIS_QTB_L,ENV Test Database,ENVTEST1,pscis,AQ$_PSCIS_QTB_L,ADDRESS#,NaN,3


Okay, now out of all the possible tables and columns we have in DEV, the table above represents all of those that have UPPERCASE column names.

The following envs and schemas are the ones that have uppercase only column names.

In [5]:
uppercase_only_df['env'].unique()

array(['ENV Test Database', 'DBQ01', 'geobc test database',
       'RRS Test Database'], dtype=object)

In [6]:
uppercase_only_df['Schema'].unique()

array(['pscis', 'fish', 'consep', 'csa', 'app_rrs', 'vgis'], dtype=object)

In [39]:
#Re-write the CSV with the new added column index column
df.to_csv('all_cols.csv')

In [7]:
# Find unique combinations between the two columns
unique_combinations = df.drop_duplicates(subset=['env', 'Database'])

# Display the unique combinations and count how many there are
print(unique_combinations)
print(f"Number of unique combinations: {len(unique_combinations)}")


                                              Table Name  \
0             DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE   
5      dlhdev.lhousedev.fdw_ods_fta_replication.aac_a...   
308        ENV Test Database.ENVTEST1.acat.ACAT_CONTACTS   
623            geobc test database.GEOTST.mascot.ACCOUNT   
5808        MEMTST1.MEMTST.app_mms_interface.APPLICATION   
6100   RRS Test Database.RRSTST1.app_rrs.APPLICATION_...   
9091   ODS.odsdev.ats_replication.ats_athn_close_reas...   
12288  Mineral Titles Online TEST Database.MTOTEST.co...   
32746  ENV Test Database.ENVTEST1.swis."Count_entered...   

                                       env   Database  \
0                                    DBQ01      DBQ01   
5                                   dlhdev  lhousedev   
308                      ENV Test Database   ENVTEST1   
623                    geobc test database     GEOTST   
5808                               MEMTST1     MEMTST   
6100                     RRS Test Database    RRSTST1   


In [8]:
# Find unique combinations and their counts
combination_counts = df.groupby(['env', 'Database']).size().reset_index(name='count')

# Create a dictionary with the combinations as keys and their counts as values
combo_dict = {(row['env'], row['Database']): row['count'] for _, row in combination_counts.iterrows()}

# Display the dictionary
print(combo_dict)

# Print each combination and its count in a vertically stacked list
for combo, count in combo_dict.items():
    print(f"Combination {combo}: {count}")

{('DBQ01', 'DBQ01'): 62149, ('ENV Test Database', 'ENVTEST1'): 45394, ('MEMTST1', 'MEMTST'): 803, ('Mineral Titles Online TEST Database', 'MTOTEST'): 2392, ('ODS', 'odsdev'): 683, ('RRS Test Database', 'RRSTST1'): 2445, ('dlhdev', 'lhousedev'): 1779, ('geobc test database', 'GEOTST'): 16917}
Combination ('DBQ01', 'DBQ01'): 62149
Combination ('ENV Test Database', 'ENVTEST1'): 45394
Combination ('MEMTST1', 'MEMTST'): 803
Combination ('Mineral Titles Online TEST Database', 'MTOTEST'): 2392
Combination ('ODS', 'odsdev'): 683
Combination ('RRS Test Database', 'RRSTST1'): 2445
Combination ('dlhdev', 'lhousedev'): 1779
Combination ('geobc test database', 'GEOTST'): 16917


In [9]:
df['env'].unique()

array(['DBQ01', 'dlhdev', 'ENV Test Database', 'geobc test database',
       'MEMTST1', 'RRS Test Database', 'ODS',
       'Mineral Titles Online TEST Database', '#SPILL!'], dtype=object)

In [10]:
df['Database'].unique()

array(['DBQ01', 'lhousedev', 'ENVTEST1', 'GEOTST', 'MEMTST', 'RRSTST1',
       'odsdev', 'MTOTEST', nan], dtype=object)

In [11]:
df['Schema'].unique()

array(['the', 'fdw_ods_fta_replication', 'acat', 'fish', 'vgis', 'mascot',
       'app_pefp', 'ems_public', 'webade', 'consep', 'sis', 'aps',
       'akharagh', 'mascot_test', 'swis', 'information_schema', 'fadm',
       'wimsi', 'ngps', 'csa', 'app_mms_interface', 'app_rrs', 'print',
       'odm', 'pscis', 'esri', 'rep_bcparks', 'ars', 'swis_archive',
       'ata', 'ats', 'fdw_ods_ats_replication', 'ats_replication',
       'test_ats_replication', 'audsys', 'wells', 'barts', 'bath',
       'bcemap', 'fdw_ods_pmt_gis_analysis', 'corp', 'bei', 'eirs',
       'ceei2', 'cbd', 'ccsd', 'ceei', 'odm_mtr', 'bcgnis', 'melp',
       'paso', 'cms', 'cms_client', 'cnar', 'mta_spatial', 'enf', 'wtr',
       'cwi_spi_dc', 'epic', nan, 'crs', 'cwi_spi_opd', 'cwi_txn', 'fiss',
       'hdms', 'cogix', 'pmt_dpl', 'pmt_sandbox', 'ems', 'epic_prod_imp',
       'lexis_replication', 'pmt_dal', 'isdut', 'fdis', 'fhas',
       'fta_replication', 'fms', 'gator', 'geomark', 'leh', 'gys', 'mta',
       'p2sry',

Okay, let's see what env/schema combos are present on the CSV of compiled secuirty classification data

In [12]:
df2 = pd.read_csv('reference_csvs/cmp.csv')
df2.head()

,Database,Schema,Table,Column,Tag,Data Type,Column Length,Nullable,Rows in Table,Notes,How to Mask,How it was Masked,Comments
0,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ALLOCATION_AMOUNT_ID,PUBLIC,NUMBER,10,N,5891.0,NaN,NaN,NaN,NaN
1,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ADJUSTMENT_REASON_CODE,PUBLIC,VARCHAR2,4,N,NaN,NaN,NaN,NaN,NaN
2,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ALLOCATION_PERIOD_ID,PUBLIC,NUMBER,10,N,NaN,NaN,NaN,NaN,NaN
3,DBP01,THE,AAC_ALLOCATION_AMOUNT,ALLOWABLE_AREA_TYPE_CODE,PUBLIC,VARCHAR2,3,N,NaN,NaN,NaN,NaN,NaN
4,DBP01,THE,AAC_ALLOCATION_AMOUNT,ALLOWABLE_CUT_TYPE_CODE,PUBLIC,VARCHAR2,3,N,NaN,NaN,NaN,NaN,NaN


In [13]:
df2['Database'].unique()

array(['DBP01', 'GEOPRD'], dtype=object)

In [14]:
df2['Schema'].unique()

array(['THE', 'ATS'], dtype=object)

Great, from the above 2 code blocks we can see that the schemas and Databases that have security classification information will not need extra formatting to match the ones with uppercase as outlined further above. This saves a bit of work.

In [15]:
# Create 'ENV' column based on 'Database' column using np.where()
df2['ENV'] = np.where(df2['Database'] == 'DBP01', 'DBQ01', 'geobc test database')

And because DEV does not contain DBP01, we will go ahead and assume we change DBP01 tpo DBQ01 to make it compatible with DEV

In [16]:
df2.head()

,Database,Schema,Table,Column,Tag,Data Type,Column Length,Nullable,Rows in Table,Notes,How to Mask,How it was Masked,Comments,ENV
0,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ALLOCATION_AMOUNT_ID,PUBLIC,NUMBER,10,N,5891.0,NaN,NaN,NaN,NaN,DBQ01
1,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ADJUSTMENT_REASON_CODE,PUBLIC,VARCHAR2,4,N,NaN,NaN,NaN,NaN,NaN,DBQ01
2,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ALLOCATION_PERIOD_ID,PUBLIC,NUMBER,10,N,NaN,NaN,NaN,NaN,NaN,DBQ01
3,DBP01,THE,AAC_ALLOCATION_AMOUNT,ALLOWABLE_AREA_TYPE_CODE,PUBLIC,VARCHAR2,3,N,NaN,NaN,NaN,NaN,NaN,DBQ01
4,DBP01,THE,AAC_ALLOCATION_AMOUNT,ALLOWABLE_CUT_TYPE_CODE,PUBLIC,VARCHAR2,3,N,NaN,NaN,NaN,NaN,NaN,DBQ01


In [17]:
df2['FQN'] = df2['ENV']+'.'+df2['Database']+'.'+df2['Schema'].str.lower()+'.'+df2['Table']+'.'+df2['Column']
df2['FQN'] = df2['FQN'].str.replace('DBP01', 'DBQ01')

In [18]:
df2['Table Name'] = df2['ENV']+'.'+df2['Database']+'.'+df2['Schema'].str.lower()+'.'+df2['Table']
df2['Table Name'] = df2['Table Name'].str.replace('DBP01', 'DBQ01')
df2['index'] = df2.groupby('Table Name').cumcount()

In [19]:
df2.head(10)

,Database,Schema,Table,Column,Tag,Data Type,Column Length,Nullable,Rows in Table,Notes,How to Mask,How it was Masked,Comments,ENV,FQN,Table Name,index
0,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ALLOCATION_AMOUNT_ID,PUBLIC,NUMBER,10,N,5891.0,NaN,NaN,NaN,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ALLO...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,0
1,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ADJUSTMENT_REASON_CODE,PUBLIC,VARCHAR2,4,N,NaN,NaN,NaN,NaN,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ADJU...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,1
2,DBP01,THE,AAC_ALLOCATION_AMOUNT,AAC_ALLOCATION_PERIOD_ID,PUBLIC,NUMBER,10,N,NaN,NaN,NaN,NaN,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ALLO...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,2
3,DBP01,THE,AAC_ALLOCATION_AMOUNT,ALLOWABLE_AREA_TYPE_CODE,PUBLIC,VARCHAR2,3,N,NaN,NaN,NaN,NaN,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOWABL...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,3
4,DBP01,THE,AAC_ALLOCATION_AMOUNT,ALLOWABLE_CUT_TYPE_CODE,PUBLIC,VARCHAR2,3,N,NaN,NaN,NaN,NaN,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOWABL...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,4
5,DBP01,THE,AAC_ALLOCATION_AMOUNT,ALLOCATION_AMOUNT,CONFIDENTIAL,NUMBER,"14,4",N,NaN,NaN,NaN,NaN,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOCATI...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,5
6,DBP01,THE,AAC_ALLOCATION_AMOUNT,ADJUSTMENT_COMMENT,CONFIDENTIAL,VARCHAR2,4000,Y,NaN,NaN,Set to NULL or Scramble,THE@masking.nrs.bcgov> alter trigger the.aac_b...,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ADJUSTME...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,6
7,DBP01,THE,AAC_ALLOCATION_AMOUNT,REVISION_COUNT,PUBLIC,NUMBER,5,N,NaN,NaN,NaN,Trigger altered.,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.REVISION...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,7
8,DBP01,THE,AAC_ALLOCATION_AMOUNT,ENTRY_USERID,PROTECTED_A,VARCHAR2,30,N,NaN,NaN,Set to D or Scramble,THE@masking.nrs.bcgov> update aac_allocation_a...,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ENTRY_US...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,8
9,DBP01,THE,AAC_ALLOCATION_AMOUNT,ENTRY_TIMESTAMP,PUBLIC,DATE,7,N,NaN,NaN,NaN,5919 rows updated.,NaN,DBQ01,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ENTRY_TI...,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,9


In [20]:
df3 = df2[['Table Name','Tag','index','Column','FQN']]

In [21]:
df3.head()

,Table Name,Tag,index,Column,FQN
0,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,PUBLIC,0,AAC_ALLOCATION_AMOUNT_ID,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ALLO...
1,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,PUBLIC,1,AAC_ADJUSTMENT_REASON_CODE,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ADJU...
2,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,PUBLIC,2,AAC_ALLOCATION_PERIOD_ID,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ALLO...
3,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,PUBLIC,3,ALLOWABLE_AREA_TYPE_CODE,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOWABL...
4,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,PUBLIC,4,ALLOWABLE_CUT_TYPE_CODE,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOWABL...


In [22]:
df.head()

,Table Name,env,Database,Schema,Table,Column,Tag,index
0,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,aac_adjustment_reason_code,Test Classification.Ignore this tag,0
1,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,description,Test Classification.Ignore this tag,1
2,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,effective_date,NaN,2
3,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,expiry_date,NaN,3
4,DBQ01.DBQ01.the.AAC_ADJUSTMENT_REASON_CODE,DBQ01,DBQ01,the,AAC_ADJUSTMENT_REASON_CODE,update_timestamp,NaN,4


In [23]:
merged_df = pd.merge(df,df3, on=['Table Name','index'])

In [24]:
# Define the mapping
mapping = {
    'PROTECTED_A': 'Data Security Classification.Protected A',
    'PROTECTED_B': 'Data Security Classification.Protected B',
    'PROTECTED_C': 'Data Security Classification.Protected C',
    'PUBLIC': 'Data Security Classification.Public',
    'CONFIDENTIAL': 'Data Security Classification.Confidential'
}

# Map values in the columns using the mapping dictionary
merged_df['Tag_x'] = merged_df['Tag_y'].map(mapping)

In [25]:
merged_df['Tag_y'].unique()

array(['PUBLIC', 'CONFIDENTIAL', 'PROTECTED_A'], dtype=object)

In [26]:
merged_df['Tag_x'].unique()

array(['Data Security Classification.Public',
       'Data Security Classification.Confidential',
       'Data Security Classification.Protected A'], dtype=object)

In [27]:
merged_df[merged_df['Table']=='AAC_ALLOCATION_AMOUNT']

,Table Name,env,Database,Schema,Table,Column_x,Tag_x,index,Tag_y,Column_y,FQN
5,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,aac_allocation_amount_id,Data Security Classification.Public,0,PUBLIC,AAC_ALLOCATION_AMOUNT_ID,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ALLO...
6,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,aac_adjustment_reason_code,Data Security Classification.Public,1,PUBLIC,AAC_ADJUSTMENT_REASON_CODE,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ADJU...
7,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,aac_allocation_period_id,Data Security Classification.Public,2,PUBLIC,AAC_ALLOCATION_PERIOD_ID,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.AAC_ALLO...
8,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,allowable_area_type_code,Data Security Classification.Public,3,PUBLIC,ALLOWABLE_AREA_TYPE_CODE,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOWABL...
9,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,allowable_cut_type_code,Data Security Classification.Public,4,PUBLIC,ALLOWABLE_CUT_TYPE_CODE,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOWABL...
10,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,allocation_amount,Data Security Classification.Confidential,5,CONFIDENTIAL,ALLOCATION_AMOUNT,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ALLOCATI...
11,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,adjustment_comment,Data Security Classification.Confidential,6,CONFIDENTIAL,ADJUSTMENT_COMMENT,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ADJUSTME...
12,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,revision_count,Data Security Classification.Public,7,PUBLIC,REVISION_COUNT,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.REVISION...
13,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,entry_userid,Data Security Classification.Protected A,8,PROTECTED_A,ENTRY_USERID,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ENTRY_US...
14,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT,DBQ01,DBQ01,the,AAC_ALLOCATION_AMOUNT,entry_timestamp,Data Security Classification.Public,9,PUBLIC,ENTRY_TIMESTAMP,DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT.ENTRY_TI...


In [29]:
len(merged_df[merged_df['Tag_y']=='CONFIDENTIAL'])

306

Looks like everything is in order so let's apply the secuirty classification tags!

In [67]:
# Example: Checking if the table exists before applying tags
for index, row in merged_df[merged_df['Tag_y']=='CONFIDENTIAL'].iterrows():
    table_fqn = row['Table Name']
    column_index = row['index']
    tag_fqn = row['Tag_x']

    # Construct the API URL for checking if the table exists
    url_check = f"https://nr-data-catalogue-test.apps.emerald.devops.gov.bc.ca/api/v1/tables/name/{table_fqn}?fields=columns,tags"

    # Send a GET request to check if the table exists
    response_check = requests.get(url_check, headers=headers_get)

    if response_check.status_code == 200:
        print(f"Table {table_fqn} exists. Proceeding to add tag.")

        # Construct the API URL
        url_fqn = f"https://nr-data-catalogue-{env}.apps.emerald.devops.gov.bc.ca/api/v1/tables/name/{table_fqn}?fields=columns,tags"

        # JSON body for adding a tag to the specific column
        data_patch = [
            {
                "op": "add",
                "path": f"/columns/{column_index}/tags",
                "value": [
                    {
                        "tagFQN": tag_fqn,
                        "labelType": "Manual",
                        "state": "Confirmed"
                    }
                ]
            }
        ]

        # Send the PATCH request to add the tag
        response = requests.patch(url_fqn, headers=headers_patch, json=data_patch)
        time.sleep(1)

        if response.status_code == 200:
            print(f"Tag added successfully for {table_fqn} - column {column_index}!")
        else:
            print(f"Failed to add tag for {table_fqn} - column {column_index}: {response.status_code}")
            print(response.json())
    else:
        print(f"Table {table_fqn} not found. Skipping this entry.")


Table DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT exists. Proceeding to add tag.
Tag added successfully for DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT - column 5!
Table DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT exists. Proceeding to add tag.
Tag added successfully for DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT - column 6!
Table DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT exists. Proceeding to add tag.
Tag added successfully for DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT - column 3!
Table DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT exists. Proceeding to add tag.
Tag added successfully for DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT - column 6!
Table DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT exists. Proceeding to add tag.
Tag added successfully for DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT - column 7!
Table DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT exists. Proceeding to add tag.
Tag added successfully for DBQ01.DBQ01.the.AAC_ALLOCATION_AMOUNT_AUDIT - column 8!
Table DBQ01.DBQ01.the.AAC_ALLOCATION_AMO